<a href="https://colab.research.google.com/github/aliffaturrohman/python_labs/blob/main/regressionAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Dataset

In [2]:
!wget Url test.csv https://raw.githubusercontent.com/vopula/predictionCalorie/refs/heads/master/test.csv
!wget Url train.csv https://raw.githubusercontent.com/vopula/predictionCalorie/refs/heads/master/train.csv
!wget Url sample_submission.csv https://raw.githubusercontent.com/vopula/predictionCalorie/refs/heads/master/sample_submission.csv

--2025-05-10 16:02:04--  http://url/
Resolving url (url)... failed: Name or service not known.
wget: unable to resolve host address ‘url’
--2025-05-10 16:02:05--  http://test.csv/
Resolving test.csv (test.csv)... failed: Name or service not known.
wget: unable to resolve host address ‘test.csv’
--2025-05-10 16:02:05--  https://raw.githubusercontent.com/vopula/predictionCalorie/refs/heads/master/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10524308 (10M) [text/plain]
Saving to: ‘test.csv.1’

test.csv.1          100%[===================>]  10.04M  --.-KB/s    in 0.1s    

2025-05-10 16:02:05 (98.0 MB/s) - ‘test.csv.1’ saved [10524308/10524308]

FINISHED --2025-05-10 16:02:05--
Total wall clock time: 0.3s
Downloaded: 1 files, 10M in 0.1s (98.0 MB/s)

**Problem to solve: predict how many calories were burned during a workout**

*For data scientist its all about reducing error in prediction.*

Correlation of column and target varible
1. Body_Temp is related to calories because the process creates heat and maintains a healthy baseline internal temperature (https://www.forhers.com/blog/how-to-raise-core-body-temperature-for-weight-loss)
2. Heart_Rate is related to calories because heart beats faster means muscle need more oxygen-rich blood and the body uses calorie to keep up with the extra oxygen needed (https://www.coospo.com/blogs/knowledge/calories-burned-by-heart-rate-understanding-the-connection?srsltid=AfmBOoqn33saM4wUE6pgDOfI-TY7ojgJB_SzwTnhLiMDI1fjusd0pn0_)
3. Duration is affecting calorie but up to one time because low to moderate lead to higher calorie burn but high activity some kind of conserving energy it has (https://magazine.scienceconnected.org/2021/03/more-exercise-doesnt-always-burn-more-calories/)
4. Weight does affect calorie because heavier person need more energy to move the body (https://www.livestrong.com/article/300582-do-fat-people-burn-more-calories/)
5. Height affect calorie because higher metabloc rate than short people (https://sportcoaching.co.nz/do-taller-people-burn-more-calories/)
6. Sex has affection to calorie because larger muscle mass (https://www.everydayhealth.com/fitness/factors-that-can-affect-how-many-calories-you-burn/)
7. Age do affect calorie because key hormone that promote protein systhesis become more resistant as getting older (https://www.everydayhealth.com/fitness/factors-that-can-affect-how-many-calories-you-burn/)

# Data Preprocessing

In [67]:
import pandas as pd

In [68]:
data_train = pd.read_csv('train.csv')
data_train.head(6)

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0
5,5,female,26,156.0,56.0,19.0,100.0,40.5,103.0


In [69]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id          750000 non-null  int64  
 1   Sex         750000 non-null  object 
 2   Age         750000 non-null  int64  
 3   Height      750000 non-null  float64
 4   Weight      750000 non-null  float64
 5   Duration    750000 non-null  float64
 6   Heart_Rate  750000 non-null  float64
 7   Body_Temp   750000 non-null  float64
 8   Calories    750000 non-null  float64
dtypes: float64(6), int64(2), object(1)
memory usage: 51.5+ MB


In [70]:
data_test = pd.read_csv('test.csv')
data_test.head(3)

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,750000,male,45,177.0,81.0,7.0,87.0,39.8
1,750001,male,26,200.0,97.0,20.0,101.0,40.5
2,750002,female,29,188.0,85.0,16.0,102.0,40.4


In [71]:
def basic_preprocessing(dataset, isDataTrain=False):
    dataset_prep = dataset.copy()
    dataset_prep.drop_duplicates(inplace=True)
    dataset_prep.drop('id', axis=1, inplace=True)
    encoded_df = pd.get_dummies(dataset_prep['Sex'], prefix='Sex', dtype=int)
    dataset_prep = pd.concat([dataset_prep, encoded_df], axis=1).drop('Sex', axis=1)
    cols_to_scale = [x for x in range(dataset_prep.shape[1] - 2)]

    dataset_prep.iloc[:, cols_to_scale] = dataset_prep.iloc[:, cols_to_scale].astype(float)
    mean = dataset_prep.iloc[:, cols_to_scale].mean()
    std = dataset_prep.iloc[:, cols_to_scale].std()
    scaled_data = dataset_prep.copy()
    scaled_data.iloc[:, cols_to_scale] = (dataset_prep.iloc[:, cols_to_scale] - mean) / std

    column_reposition = ['Sex_female', 'Sex_male', 'Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']

    if isDataTrain:
        column_reposition.append('Calories')
        scaled_data = scaled_data[column_reposition]
        return scaled_data, std, mean
    else:
        scaled_data = scaled_data[column_reposition]
        return scaled_data

In [72]:
data_train, std, mean = basic_preprocessing(data_train, isDataTrain=True)
data_test = basic_preprocessing(data_test)

<ipython-input-71-34dd4d31e956>:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0         36.0
1         64.0
2         51.0
3         20.0
4         38.0
          ... 
749995    28.0
749996    64.0
749997    60.0
749998    45.0
749999    39.0
Name: Age, Length: 750000, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataset_prep.iloc[:, cols_to_scale] = dataset_prep.iloc[:, cols_to_scale].astype(float)
<ipython-input-71-34dd4d31e956>:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0         45.0
1         26.0
2         29.0
3         39.0
4         30.0
          ... 
249995    56.0
249996    32.0
249997    31.0
249998    62.0
249999    31.0
Name: Age, Length: 250000, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataset_prep

# Modelling

## Linear Regression

In [73]:
from sklearn.model_selection import train_test_split

X = data_train.drop('Calories', axis=1)
y = data_train['Calories']

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [74]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [75]:
calories_predict_reg = model.predict(X_val)


In [76]:
from sklearn.metrics import mean_squared_error
metrics = mean_squared_error(calories_predict_reg, y_val)

In [77]:
metrics

0.03141299742367859

In [78]:
def inverse_transform(scaled_data, std, mean):
    std = std.to_numpy() if isinstance(std, pd.Series) else std
    mean = mean.to_numpy() if isinstance(mean, pd.Series) else mean

    std = std.reshape(1, -1)
    mean = mean.reshape(1, -1)

    original_data = scaled_data * std + mean
    return original_data.ravel()

In [79]:
calories_ori = inverse_transform(calories_predict_reg, std['Calories'], mean['Calories'])

In [80]:
calories_ori

array([197.08686251,  67.31936343,  41.13220736, ...,  44.47120989,
        27.6725451 , 166.11530054])

In [83]:
pred = model.predict(data_test)

In [84]:
ori_pred = inverse_transform(pred, std['Calories'], mean['Calories'])

In [95]:
ori_pred = np.round(ori_pred, 3)

In [96]:
import numpy as np
df_result = pd.DataFrame({
    'id': np.arange(75000, 75000 + len(ori_pred)),
    'Calories': ori_pred
})
df_result.set_index("id", inplace=True)

In [97]:
df_result.head()

,Calories
id,
75000,21.418
75001,114.722
75002,93.251
75003,130.843
75004,73.693


In [98]:
df_result.to_csv("submission.csv", sep=",")

In [99]:
pd.read_csv("submission.csv").head()

,id,Calories
0,75000,21.418
1,75001,114.722
2,75002,93.251
3,75003,130.843
4,75004,73.693
